## Notebook Datos

In [40]:
import pandas as pd
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import geojson
import geopandas as gpd
from bs4 import BeautifulSoup 
import io
import zipfile
from mpl_toolkits.axes_grid1 import make_axes_locatable

* # Veamos la tabla del torneo nacional de **Futbol** en el 2010 ⚽

In [41]:
url = 'https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Chile_2010'
respuesta = requests.get(url).text

soup = BeautifulSoup(respuesta)

In [42]:
info = soup.find('body',{'class':'mediawiki ltr sitedir-ltr mw-hide-empty-elt ns-0 ns-subject mw-editable page-Primera_División_de_Chile_2010 rootpage-Primera_División_de_Chile_2010 skin-vector action-view skin-vector-legacy'})
tabla = info.find('div',{'id':'content'}).find('div',{'id':'mw-content-text'}).find('div',{'class':'mw-parser-output'}).find_all('table',{'class':'wikitable'})[3].find('tbody')
equipos = tabla.find_all('tr')[1::]
aux = []
df = pd.DataFrame(columns=['POS','Equipo','pts','j','g','e','p','gf','gc','dif'])
for rows in equipos:
    columna = rows.find_all('td')
    col = [columna[i].text.strip() for i in range(10)]
    aux.append(col)
df = pd.DataFrame(aux, columns=['POS','Equipo','pts','j','g','e','p','gf','gc','dif'])
df.set_index(['POS'])

,Equipo,pts,j,g,e,p,gf,gc,dif
POS,,,,,,,,,
1.,Universidad Católica,74,34,23,5,6,77,39,+38
2.,Colo-Colo 1,71,34,22,5,7,67,34,+33
3.,Audax Italiano,65,34,20,5,9,75,58,+17
4.,Universidad de Chile,64,34,20,4,10,75,48,+27
5.,Unión Española,52,34,14,10,10,58,50,+8
6.,Huachipato,48,34,12,12,10,44,40,+4
7.,Deportes La Serena,45,34,13,6,15,45,59,-14
8.,Santiago Wanderers,45,34,12,9,13,48,52,-4
9.,Unión San Felipe,43,34,12,7,15,38,47,-9


* # Veamos los ingresos

Obtengamos de TransferMarkt los datos de ingresos y salidas economicas por fichajes

In [43]:
transferdf=pd.read_csv('transferencias_2010.csv')
transferdf

,Equipos,Ingresos,Gastos,Balance Total,I.E
0,Universidad Católica,245.000,145.000,100.000,€
1,Colo-Colo,7.07.0000,1.980.000,5.090.000,€
2,Audax Italiano,7.100.000,0,7.100.000,€
3,Santiago Morning,350.000,0,350.000,€
4,Everton,1.230.000,0,1.230.000,€
5,San Luis,0,0,0,€


### Datos de los clubes 2010

In [93]:
tabla =info.find(id='content').find(id='bodyContent').find(id='mw-content-text').find('div',{'class':'mw-parser-output'}).find('table',border="0").find('tbody').find('tr').find_all('td')[0].find('table').find_all('tr')[1::]
datosequipo=[]
for equipos in tabla:
    data=[]
    for i in range(6):
        dato = equipos.find_all('td')[i].text.strip('\n')
        
        data.append(dato)
    data.pop(1)
    datosequipo.append(data)
datos_equipos_df = pd.DataFrame(datosequipo, columns=['Equipo', 'Entrenador', 'Ciudad', 'Estadio', 'Capacidad Estadio'])
datos_equipos_df



,Equipo,Entrenador,Ciudad,Estadio,Capacidad Estadio
0,Audax Italiano,Omar Labruna,Santiago,Bicentenario de La Florida,12.000
1,Cobreloa,Mario Soto,Calama,Municipal de Calama,13.000
2,Cobresal,Luis Musrri,El Salvador,El Cobre,15.000
3,Colo Colo,Diego Cagna,Santiago,Monumental David Arellano,47.017
4,Deportes La Serena,Víctor Hugo Castañeda,La Serena,La Portada,14.000
5,Everton,Diego Osella (interino),Viña del Mar,Sausalito,18.037
6,Huachipato,Arturo Salah,Talcahuano,CAP,10.500
7,Ñublense,Luis Marcoleta,Chillán,Bicentenario Nelson Oyarzún Arenas,12.000
8,O'Higgins,Marco Antonio Figueroa,Rancagua,El Teniente,14.450
9,Palestino,Gustavo Benítez,Santiago,Municipal de La Cisterna,12.000
